In [1]:
using JSON
using CSV 
using XLSX
using DataFrames

In [4]:
df = CSV.read("flights.csv", DataFrame;  delim='#');

In [5]:
df = dropmissing(df)

Row,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED
,Int64,Int64,Int64,Int64,String3,Int64,String7,String7,String7,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Int64,Float64,Float64,Int64,Int64
1,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0
2,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0
3,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0
4,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0
5,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0
6,2015,1,1,4,DL,806,N3730B,SFO,MSP,25,20.0,-5.0,18.0,38.0,217.0,230.0,206.0,1589,604.0,6.0,602,610.0,8.0,0,0
7,2015,1,1,4,NK,612,N635NK,LAS,MSP,25,19.0,-6.0,11.0,30.0,181.0,170.0,154.0,1299,504.0,5.0,526,509.0,-17.0,0,0
8,2015,1,1,4,US,2013,N584UW,LAX,CLT,30,44.0,14.0,13.0,57.0,273.0,249.0,228.0,2125,745.0,8.0,803,753.0,-10.0,0,0
9,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,19.0,-11.0,17.0,36.0,195.0,193.0,173.0,1464,529.0,3.0,545,532.0,-13.0,0,0


In [13]:
airline = "NK"
for m in 1:2
    df_DL = filter(row -> row.MONTH == m && row.AIRLINE == airline, df)
    df_DL = df_DL[:, ["YEAR", "MONTH", "DAY", "TAIL_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "DEPARTURE_TIME", "AIR_TIME", "ARRIVAL_TIME"]]
    df_DL[!, :DEPARTURE_TIME] = @. string(floor(Int, round(Int, df_DL.DEPARTURE_TIME) ÷ 100), ":", lpad(round(Int,df_DL.DEPARTURE_TIME) % 100, 2, '0'))
    df_DL[!, :ARRIVAL_TIME] = @. string(floor(Int, round(Int,df_DL.ARRIVAL_TIME) ÷ 100), ":", lpad(round(Int,df_DL.ARRIVAL_TIME) % 100, 2, '0'))
    XLSX.writetable(airline*"_2015-0"*string(m)*"_real_flights.xlsx", df_DL, overwrite = true)
end


In [9]:
airline = "AS"
for m in 6:12
    df_temp = filter(row -> row.MONTH == m && row.AIRLINE == airline, df)
    df_temp = df_temp[:, ["YEAR", "MONTH", "DAY", "TAIL_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "DEPARTURE_TIME", "AIR_TIME", "ARRIVAL_TIME"]]
    for row in eachrow(df_temp)
        dt = (round(Int, row.DEPARTURE_TIME)÷100)*60 + 1440*(row.DAY-1) + round(Int,row.DEPARTURE_TIME) % 100
        at = (round(Int, row.ARRIVAL_TIME)÷100)*60 + 1440*(row.DAY-1) + round(Int,row.ARRIVAL_TIME) % 100
        if at < dt
            at += 1440
        end 
        row.DEPARTURE_TIME = dt
        row.ARRIVAL_TIME = at
    end 
    XLSX.writetable(airline*"_2015-0"*string(m)*"_real_fl.xlsx", df_temp, overwrite = true)
end

In [ ]:

println(unique(df.AIRLINE))
